In [ ]:
'''
"2024-01-11-20-02-45" => ScorePredictor => ScoreMultiPairH5Dataset
"2023-10-28-18-33-37" => PoseRefinePredictor => RefineNet
'''

In [1]:
import torch
import os
import yaml
from omegaconf import OmegaConf
from learning.models.score_network import ScoreNetMultiPair
from learning.models.refine_network import RefineNet
# Assuming the configuration and model checkpoint paths are correct
root_path = 'weights/2023-10-28-18-33-37/'
config_path = root_path + 'config.yml'
model_path = root_path + 'model_best.pth'

# Load configuration using OmegaConf
cfg = OmegaConf.load(config_path)

# Adjust the configuration as needed for loading
cfg['ckpt_dir'] = model_path
cfg['enable_amp'] = True

# Make sure this matches the configuration used during training
cfg['c_in'] = 6  # Set this according to your checkpoint details

# Initialize the model
print("c_in before model initialization:", cfg['c_in'])
# model = ScoreNetMultiPair(cfg=cfg, c_in=cfg['c_in']).cuda()
model = RefineNet(cfg=cfg, c_in=cfg['c_in']).cuda()

# Load model weights
ckpt = torch.load(cfg['ckpt_dir'])
if 'model' in ckpt:
    ckpt = ckpt['model']
model.load_state_dict(ckpt)

model.eval()  # Set the model to evaluation mode

# Quantization
# model_quantized = torch.quantization.quantize_dynamic(
#     model,
#     {torch.nn.Linear, torch.nn.BatchNorm2d},  # Adjust based on which layers you want to quantize
#     dtype=torch.qint8
# )

model_quantized = torch.quantization.quantize_dynamic(
    model,
    {torch.nn.Linear, torch.nn.BatchNorm2d, torch.nn.Conv2d, torch.nn.ReLU, torch.nn.MultiheadAttention},
    dtype=torch.qint8
)

# Save the quantized model
torch.save(model_quantized.state_dict(), root_path + 'model_quantized.pth')
print(f"Quantization complete and model saved - {root_path}")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Warp CUDA error 2: out of memory (in function cuda_device_get_memory_info, /buildAgent/work/a9ae500d09a78409/warp/native/warp.cu:1744)
Warp CUDA error 201: invalid device context (in function cuda_device_get_memory_info, /buildAgent/work/a9ae500d09a78409/warp/native/warp.cu:1747)
Warp CUDA error 201: invalid device context (in function cuda_device_get_memory_info, /buildAgent/work/a9ae500d09a78409/warp/native/warp.cu:1749)


Warp 1.0.2 initialized:
   CUDA Toolkit 11.5, Driver 12.2
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:1"   : "NVIDIA A100-SXM4-40GB" (0 GiB, sm_80, mempool enabled)
     "cuda:2"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
     "cuda:3"   : "NVIDIA A100-SXM4-40GB" (39 GiB, sm_80, mempool enabled)
   CUDA peer access:
     Supported fully (all-directional)
   Kernel cache:
     /home/aagrawal/.cache/warp/1.0.2
c_in before model initialization: 6
Quantization complete and model saved - weights/2023-10-28-18-33-37/


In [1]:
import torch

# Assuming the model class is already imported and named 'ExampleModel'
model = torch.load('/nfs/aiteam/arjun/Development/leap/FoundationPose/weights/2024-01-11-20-02-45/model_best.pth')
print(model.conv1.in_channels)

AttributeError: 'dict' object has no attribute 'conv1'

# ONNX torch quantisation 